In [79]:
import pandas as pd

In [80]:
df = pd.read_csv('spotify_millsongdata.csv')
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [81]:
df.shape

(57650, 4)

In [82]:
#check for missing values
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [83]:
df = df.sample(5000).drop('link', axis=1).reset_index(drop=True)
df.head()

,artist,song,text
0,Ween,Mutilated Lips,I lick my brain in silence \r\nRather squeeze...
1,Kiss,Do You Love Me,You really like my limousine \r\nYou like the...
2,Hank Snow,I Keep Dreaming Of You All The Time,In my dreams an old friend came to see you \r...
3,Prince,Bambi,I knew from the start \r\nThat I loved you wi...
4,Warren Zevon,Searching For A Heart,Darkness in the morning \r\nShadows on the la...


In [85]:
df.tail(10)

,artist,song,text
4990,Howard Jones,Dream Into Action,Between every man there is a division \r\nNob...
4991,Louis Armstrong,I Got Rhythm,I got rhythm....I got music....I got my gal \...
4992,Frank Sinatra,Change Partners,Must you dance every dance with the same fortu...
4993,Fleetwood Mac,Beautiful Child,Beautiful child \r\nBeautiful child \r\nYou ...
4994,Bread,Truckin',Truckin' down the highway \r\nGet outta my wa...
4995,Widespread Panic,Contentment Blues,I've got no hard lipped woman \r\nNagging at ...
4996,Santana,Let Me Love You Tonight,It's midnight - And I don't want to see you le...
4997,Isley Brothers,Said Enough (Featuring Jill Scott),[Jill:] \r\nThere's so many things I want to ...
4998,Noa,She Went To The River,"She went to the river, \r\nShe never thought ..."
4999,Judas Priest,Cheater,"I got home late this evening, stumbled up the ..."


## Text Preprocessing/Cleaning

In [86]:
df['text'][0]

"I lick my brain in silence  \r\nRather squeeze my head instead  \r\nMidget man provoking violence  \r\nListen not to what I said  \r\n  \r\nI said please calm it down  \r\nEverything is turning brown  \r\n  \r\nMutilated lips give a kiss on the wrist  \r\nOf the worm like tips of tentacles expanding  \r\nIn my mind, I'm fine, accepting only fresh brine  \r\nYou can get another drop of this, yeah you wish  \r\n  \r\nMutilated lips give a kiss on the wrist  \r\nOf the worm like tips of tentacles expanding  \r\nIn my mind, I'm fine, accepting only fresh brine  \r\nYou can get another drop of this, yeah you wish  \r\n  \r\nLaughing lady living lover  \r\nOoo you sassy frassy lassie  \r\nFind me the skull of Haile Sellase, I  \r\nGive me shoes so I can tapsy  \r\nTap all over this big world  \r\nTake my hand you ugly girl  \r\n  \r\nMutilated lips give a kiss on the wrist  \r\nOf the worm like tips of tentacles expanding  \r\nIn my mind, I'm fine, accepting only fresh brine  \r\nYou can ge

In [87]:
import re

In [88]:
#change to lowercase,replace all the non-word charaters(not letter,figures or underscore)and the \nto white spaces
df['text'] = df['text'].str.lower().replace(r'^\W\s',' ').replace(r'\n',' ', regex=True)
df['text'][0]

"i lick my brain in silence  \r rather squeeze my head instead  \r midget man provoking violence  \r listen not to what i said  \r   \r i said please calm it down  \r everything is turning brown  \r   \r mutilated lips give a kiss on the wrist  \r of the worm like tips of tentacles expanding  \r in my mind, i'm fine, accepting only fresh brine  \r you can get another drop of this, yeah you wish  \r   \r mutilated lips give a kiss on the wrist  \r of the worm like tips of tentacles expanding  \r in my mind, i'm fine, accepting only fresh brine  \r you can get another drop of this, yeah you wish  \r   \r laughing lady living lover  \r ooo you sassy frassy lassie  \r find me the skull of haile sellase, i  \r give me shoes so i can tapsy  \r tap all over this big world  \r take my hand you ugly girl  \r   \r mutilated lips give a kiss on the wrist  \r of the worm like tips of tentacles expanding  \r in my mind, i'm fine, accepting only fresh brine  \r you can get another drop of this, yeah

After regex,now we need to transform the data into tokens then later into tokens. Token techniques includes bag of words, back to back and tf-idf. But before that we need to remove the stop words.This is called Tokenization. Then we need to convert into one vector in a process called Vectorisation. Then calculate th distance of each vector. To reccomend is based on the distance. This instance is not for Eucladian distance but the Theta (Cosine Similarity)tag base
. This will create a Content-Based Recommender. There are different types of recommender system such as the Collaborative Filtering which is based on the user ratings and there is the Population, based on majority. And another called by Hybrid which is both tag base and popularity base such as Youtube or Alibaba.

In [89]:
import nltk
from nltk.stem.porter import PorterStemmer

In [90]:
stemmer = PorterStemmer()

In [91]:
def token(txt):
    token = nltk.word_tokenize(txt)
    a = [stemmer.stem(w) for w in token]
    return " ".join(a)

In [92]:
token("you are beautiful, beauty")

'you are beauti , beauti'

In [93]:
df['text'].apply(lambda x: token(x))

0       i lick my brain in silenc rather squeez my hea...
1       you realli like my limousin you like the way t...
2       in my dream an old friend came to see you righ...
3       i knew from the start that i love you with all...
4       dark in the morn shadow on the land certain in...
                              ...                        
4995    i 've got no hard lip woman nag at who i shoul...
4996    it 's midnight - and i do n't want to see you ...
4997    [ jill : ] there 's so mani thing i want to sa...
4998    she went to the river , she never thought twic...
4999    i got home late thi even , stumbl up the stair...
Name: text, Length: 5000, dtype: object

In [94]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [95]:
tfid = TfidfVectorizer(analyzer = 'word', stop_words='english')

In [96]:
matrix = tfid.fit_transform(df['text'])

In [97]:
cosine_similarity(matrix)

array([[1.        , 0.00953706, 0.04499882, ..., 0.0472989 , 0.01231488,
        0.00407474],
       [0.00953706, 1.        , 0.10737958, ..., 0.15030841, 0.00624552,
        0.00888095],
       [0.04499882, 0.10737958, 1.        , ..., 0.14900553, 0.01992319,
        0.01997761],
       ...,
       [0.0472989 , 0.15030841, 0.14900553, ..., 1.        , 0.0147451 ,
        0.05712152],
       [0.01231488, 0.00624552, 0.01992319, ..., 0.0147451 , 1.        ,
        0.00511591],
       [0.00407474, 0.00888095, 0.01997761, ..., 0.05712152, 0.00511591,
        1.        ]])

In [98]:
similar = cosine_similarity(matrix)
similar[0]

array([1.        , 0.00953706, 0.04499882, ..., 0.0472989 , 0.01231488,
       0.00407474])

In [104]:
df[df['song']=='Let Me Love You Tonight'].index[0]

4996

### RECOMMENDER FUNCTION

In [108]:
def recommender(song_name):
    idx = df[df['song']==song_name].index[0]
    distance = sorted(list(enumerate(similar[idx])), reverse=True, key=lambda x:x[1])
    song = []
    for s_id in distance[1:5]:
        song.append(df.iloc[s_id[0]].song)
    return song

In [109]:
recommender("Let Me Love You Tonight")

['Again Tonight', "Let's Go Tonight", 'All Tore Up', 'Tonight']

In [111]:
import pickle

In [112]:
pickle.dump(similar, open("similarity","wb"))

In [113]:
pickle.dump(df, open("df","wb"))